In [17]:
import sys
sys.path.append('..')
from src.data_processing import (
    excel_to_dataframe,
    read_csv_to_list
)

from src.modelling import (
    run_model,
    results_to_dataframe,
    get_recovery_rate_vector,
    get_cdr_vector,
    get_cpr_vector,
    display_positive_cashflows
)
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)


import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
file_path = "../data/2024-Case.xlsx"
sheet_static = "DATA-Static" 
sheet_month_end = "DATA-Month End Balances"
sheet_payments_due = "DATA-Payment Due" 
sheet_payments_made = "DATA-Payment Made"

cpr_sheet = "../data/cpr.csv"
cdr_sheet = "../data/cdr.csv"
recovery_sheet = "../data/recovery.csv"

MAX_TERM = 180  # Assuming a maximum loan term of 15 years


# Q9

### Model Overview

`modelling.py` implements a flexible cashflow model for loan portfolios.

- The code is structured to accommodate different scenarios using different CPR (Conditional Prepayment Rate) and CDR (Conditional Default Rate) vectors.
- A `scenario` function can also be passed to the `run_model` function, allowing for dynamic adjustments to CPR/CDR based on specific conditions. This is particurlarly useful when the CPR/CDR curves are not static or defined in advance.


1. **Cashflow Calculation**: Computes monthly cashflows for individual loans, considering:
   - Principal and interest payments
   - Prepayments (CPR)
   - Defaults (CDR)
   - Recoveries

2. **Model Execution**: The `run_model` function processes each loan, applying:
   - Pre-reversion and post-reversion rates
   - Scenario-specific CPR/CDR adjustments

3. **Results Processing**: Converts model outputs into a pandas DataFrame, aligning cashflows from different loans based on their origination dates.

#### Scenario Customization

- A `scenario` function can be used to apply shocks or adjustments to CPR/CDR based on loan-specific characteristics or market conditions.

#### Aggregation and Analysis

- Functions are provided to aggregate cashflows across all loans.
- The resulting DataFrame enables further analysis and visualization of portfolio-level cashflows.


In [19]:
df_static = excel_to_dataframe(file_path, sheet_static, header_start=(2,1))

In [20]:
cpr_vector = np.full(180, 0.5)  # 5% CPR for all months
cdr_vector = np.full(180, 0.5)  # 2% CDR for all months
recovery_rate = 0.5
recovery_lag = 6

In [21]:
results = run_model(df_static, cpr_vector, cdr_vector, recovery_rate, recovery_lag)
results_df = results_to_dataframe(results, df_static)

In [22]:
results_df.head().T

,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31
Loan_1,0.000000,0.000000,0.000000,0.000000,0.000000
Loan_2,0.000000,0.000000,0.000000,0.000000,0.000000
Loan_3,0.000000,0.000000,0.000000,0.000000,0.000000
Loan_4,0.000000,0.000000,0.000000,0.000000,0.000000
Loan_5,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...
Loan_1627,0.000000,0.000000,0.000000,0.000000,0.000000
Loan_1628,0.000000,0.000000,0.000000,0.000000,0.000000
Loan_1629,0.000000,0.000000,0.000000,0.000000,0.000000
Loan_1630,0.000000,0.000000,0.000000,0.000000,0.000000


# Q 10

In [23]:
#1 Base-case: Using the CPR/CDR/Recovery vectors sized in Q3-Q5		

In [24]:
cpr_vector = read_csv_to_list(cpr_sheet)
cpr_vector = np.concatenate([cpr_vector, [0]* (MAX_TERM - len(cpr_vector))])

cdr_vector = read_csv_to_list(cpr_sheet)
cdr_vector = np.concatenate([cdr_vector, [0]* (MAX_TERM - len(cdr_vector))])

recovery_rate = read_csv_to_list(recovery_sheet)
recovery_rate = np.concatenate([recovery_rate, [0]* (MAX_TERM - len(recovery_rate))])
recovery_lag = 6

Successfully read 84 values from column 1 in ../data/cpr.csv
Successfully read 84 values from column 1 in ../data/cpr.csv
Successfully read 24 values from column 1 in ../data/recovery.csv


In [25]:
resultsBC1 = run_model(df_static, cpr_vector, cdr_vector, recovery_rate, recovery_lag)


In [26]:
results_dfBC1 = results_to_dataframe(resultsBC1, df_static)
results_dfBC1.head().T

,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31
Loan_1,0.000000,0.000000,0.000000,0.000000,0.000000
Loan_2,0.000000,0.000000,0.000000,0.000000,0.000000
Loan_3,0.000000,0.000000,0.000000,0.000000,0.000000
Loan_4,0.000000,0.000000,0.000000,0.000000,0.000000
Loan_5,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...
Loan_1627,0.000000,0.000000,0.000000,0.000000,0.000000
Loan_1628,0.000000,0.000000,0.000000,0.000000,0.000000
Loan_1629,0.000000,0.000000,0.000000,0.000000,0.000000
Loan_1630,0.000000,0.000000,0.000000,0.000000,0.000000


In [27]:
# 2. Base-case but CPR is 2x Post-Reversion	
cpr_vector = get_cpr_vector(0.1)
cdr_vector = get_cdr_vector(0.02)
recovery_rate = get_recovery_rate_vector(0.5)
resultsBC2 = run_model(df_static, cpr_vector, cdr_vector, recovery_rate, recovery_lag, cpr_multiplier=2)


In [28]:
results_dfBC2 = results_to_dataframe(resultsBC2, df_static)
results_dfBC2.head().T

,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31
Loan_1,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_2,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_3,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_4,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_5,0.000000,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...
Loan_1627,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_1628,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_1629,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_1630,0.000000,0.000000,0.000000,0.000000,0.00000


In [29]:
# 3. Base-case but CPR is 2x Post-Reversion on Product 2 Borrowers
cpr_vector = get_cpr_vector(0.1)
cdr_vector = get_cdr_vector(0.02)
recovery_rate = get_recovery_rate_vector(0.5)

def scenario(cpr_vector, row, pre_reversion_months):
    if row['product'] == 2:
        cpr_vector[pre_reversion_months:] = cpr_vector[pre_reversion_months:] * 2
        return cpr_vector
    else:
        return cpr_vector

resultsBC3 = run_model(df_static, cpr_vector, cdr_vector, recovery_rate, recovery_lag, scenario=scenario)


In [30]:
results_dfBC3 = results_to_dataframe(resultsBC3, df_static)
results_dfBC3.head().T

,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31
Loan_1,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_2,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_3,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_4,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_5,0.000000,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...
Loan_1627,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_1628,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_1629,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_1630,0.000000,0.000000,0.000000,0.000000,0.00000


In [31]:
# 4. Base-case but 100% CPR on Product 1 Borrowers at the month of reversion (i.e. no increased interest payments are made)				

In [32]:
cpr_vector = get_cpr_vector(0.1)
def scenario(cpr_vector, row, pre_reversion_months):
    if row['product'] == 1:
        cpr_vector[pre_reversion_months:] = [1]*len(cpr_vector[pre_reversion_months:])
        return cpr_vector
    else:
        return cpr_vector
resultsBC4 = run_model(df_static, cpr_vector, cdr_vector, recovery_rate, recovery_lag, scenario=scenario)

In [33]:
results_dfBC4 = results_to_dataframe(resultsBC4, df_static)
results_dfBC4.head().T

,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31
Loan_1,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_2,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_3,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_4,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_5,0.000000,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...
Loan_1627,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_1628,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_1629,0.000000,0.000000,0.000000,0.000000,0.00000
Loan_1630,0.000000,0.000000,0.000000,0.000000,0.00000


In [34]:
# 5. Base-case but Recoveries occur linearly over 24 months post-default.				

# Q 11

### Scenario: Conditional Prepayment Rate (CPR) approaches 100% (0.999)
The model should show an immediate or near-immediate prepayment of all loans. This is because the CPR represents the proportion of loans that are expected to be paid off early. When CPR is close to 1, almost all loans are expected to be prepaid in the next period.

Cash flows should be concentrated in a single period or very few initial periods.

In [35]:
cpr_vector = get_cpr_vector(0.99999)
cdr_vector = get_cdr_vector(0.02)
recovery_rate = get_recovery_rate_vector(0.5)
recovery_lag = 6

In [36]:
results = run_model(df_static, cpr_vector, cdr_vector, recovery_rate, recovery_lag)
results_df = results_to_dataframe(results, df_static)

In [37]:
results_df[results_df["Loan_1085"]> 0][["Loan_1085"]]

,Loan_1085
2018-05-31,79695.529718
2018-06-30,30664.933531
2018-07-31,11793.476703
2018-11-30,38.920243
2018-12-31,13.308939
2019-01-31,3.451367


In [38]:
display_positive_cashflows(results_df)

Displaying cashflows > 0 for 5 loans:
                  Loan_1        Loan_2        Loan_3        Loan_4  \
2017-04-30      0.000000      0.000000      0.000000  81263.774773   
2017-05-31      0.000000      0.000000      0.000000  31261.291409   
2017-06-30      0.000000      0.000000      0.000000  12020.419764   
2017-10-31      0.000000      0.000000      0.000000     39.689084   
2017-11-30      0.000000      0.000000      0.000000     13.570111   
2017-12-31      0.000000      0.000000      0.000000      3.519574   
2018-02-28      0.000000      0.000000  58930.338164      0.000000   
2018-03-31      0.000000      0.000000  22676.469867      0.000000   
2018-04-30      0.000000      0.000000   8721.687448      0.000000   
2018-08-31      0.000000      0.000000     28.778694      0.000000   
2018-09-30      0.000000      0.000000      9.841359      0.000000   
2018-10-31      0.000000      0.000000      2.552029      0.000000   
2020-07-31      0.000000  68302.644014      0.000000

### Scenario: Conditional Prepayment Rate (CPR) approaches 0% (0.0001)
Cash flows should closely match the original amortization schedule. The outstanding balance should decline at the rate defined by the original loan terms.


In [39]:
cpr_vector = get_cpr_vector(0.00001)
cdr_vector = get_cdr_vector(0.02)
recovery_rate = get_recovery_rate_vector(0.5)
recovery_lag = 6
results = run_model(df_static, cpr_vector, cdr_vector, recovery_rate, recovery_lag)
results_df = results_to_dataframe(results, df_static)

In [40]:
results_df[results_df["Loan_1085"]> 0][["Loan_1085"]]

,Loan_1085
2018-05-31,5186.237615
2018-06-30,5186.233367
2018-07-31,5186.229116
2018-08-31,5186.224862
2018-09-30,5186.220605
2018-10-31,5186.216344
2018-11-30,5287.799949
2018-12-31,5283.508366
2019-01-31,5279.213631
2019-02-28,5274.915743


In [41]:
display_positive_cashflows(results_df)

Displaying cashflows > 0 for 5 loans:
            Loan_1  Loan_2  Loan_3       Loan_4       Loan_5
2017-04-30     0.0     0.0     0.0  5282.605841     0.000000
2017-05-31     0.0     0.0     0.0  5282.601504     0.000000
2017-06-30     0.0     0.0     0.0  5282.597165     0.000000
2017-07-31     0.0     0.0     0.0  5282.592823     0.000000
2017-08-31     0.0     0.0     0.0  5282.588479     0.000000
...            ...     ...     ...          ...          ...
2023-01-31     0.0     0.0     0.0     0.000000  5993.119335
2023-02-28     0.0     0.0     0.0     0.000000  5988.116744
2023-03-31     0.0     0.0     0.0     0.000000  5983.114265
2023-04-30     0.0     0.0     0.0     0.000000  5978.111897
2023-05-31     0.0     0.0     0.0     0.000000  5973.109639

[69 rows x 5 columns]


### Scenario: Conditional Default Rate (CDR) approaches 100% (0.9999)
The model should show an immediate or near-immediate default of all loans.
We can see the loans default and recoveries kick in.

In [43]:
cpr_vector = get_cpr_vector(0.5)
cdr_vector = get_cdr_vector(0.999999)
recovery_rate = get_recovery_rate_vector(0.5)
recovery_lag = 6
results = run_model(df_static, cpr_vector, cdr_vector, recovery_rate, recovery_lag)
results_df = results_to_dataframe(results, df_static)

In [44]:
results_df[results_df["Loan_1085"]> 0][["Loan_1085"]]

,Loan_1085
2018-05-31,11965.221978
2018-06-30,6934.864291
2018-07-31,5421.247361
2018-11-30,38976.697701
2018-12-31,10054.397654
2019-01-31,1351.779325
2020-05-31,3081.793149
2020-06-30,1258.524154
2020-07-31,701.021494
2020-08-31,530.553386


In [45]:
display_positive_cashflows(results_df)

Displaying cashflows > 0 for 5 loans:
                  Loan_1        Loan_2        Loan_3        Loan_4  \
2017-04-30      0.000000      0.000000      0.000000  12195.504106   
2017-05-31      0.000000      0.000000      0.000000   7065.472987   
2017-06-30      0.000000      0.000000      0.000000   5522.342382   
2017-10-31      0.000000      0.000000      0.000000  39746.653993   
2017-11-30      0.000000      0.000000      0.000000  10251.276432   
2017-12-31      0.000000      0.000000      0.000000   1378.967710   
2018-02-28      0.000000      0.000000   8848.692296      0.000000   
2018-03-31      0.000000      0.000000   5129.171489      0.000000   
2018-04-30      0.000000      0.000000   4009.880342      0.000000   
2018-08-31      0.000000      0.000000  28820.438538      0.000000   
2018-09-30      0.000000      0.000000   7434.862499      0.000000   
2018-10-31      0.000000      0.000000    999.440529      0.000000   
2019-04-30      0.000000      0.000000      0.000000